In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 31.5MB/s]


In [3]:
city = 'San Francisco'

# Create empty list to store scraped data
datatoget = []

for i in range(0,8):    

    # Set url
    url = f'https://sfbay.craigslist.org/search/sfc/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n================')
    print(f'This is page {i+1}')
    print('================')

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):

        # Scrape dates
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            else:
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan

        # Scrape titles
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']

        # Scrape prices
        try:
            prices = link.find_all('span', class_='priceinfo')[0].text

        except:
            prices = np.nan

        #Scrape neighborhoods
        try:
            neighborhoods = link.find('div', class_='supertitle').text
        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)

        sleep(randint(4,6))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')

        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan
        
        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan

        try:
            amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
            # List comprehension used to make sure no empty strings get added to the list
            all_amenities = [am for am in amenities if am != '']
        
        except:
              all_amenities = np.nan

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities if all_amenities == all_amenities else "not available"}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This is page 1

Listing Number: 1
Square Footage: not available
Price: $3,300
Title: EPIC REA -840 Sq ft Fresh Paint/New Flooring-2 bd/1ba Condo w/PKG
Date: 1/18
Number of Bedrooms: 2br
Neighborhood: potrero hill
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-epic-rea-840-sq-ft-fresh/7579745847.html
Amenities: ['application fee details: $45 - tenant screening & processing', 'cats are OK - purrr', 'dogs are OK - wooof', 'condo', 'laundry in bldg', 'no smoking', 'attached garage', 'rent period: monthly']

Listing Number: 2
Square Footage: not available
Price: $3,500
Title: Golden Gate Views
Date: 1/18
Number of Bedrooms: 2br
Neighborhood: russian hill
Number of Bathrooms: 2
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-golden-gate-views/7579745842.html
Amenities: ['apartment', 'laundry in bldg', 'no smoking', 'carport', 'rent period: monthly']

Listing Number: 3
Square Footage: not available
Price: $16,500
Title: House-like condo with view


Listing Number: 19
Square Footage: 650
Price: $2,750
Title: 24th & Diamond charm
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: noe valley
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-24th-diamond-charm/7571192316.html
Amenities: ['apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly']

Listing Number: 20
Square Footage: 460
Price: $2,495
Title: 1495 7th/Beautiful Studio/Gift card =2 Months Rent
Date: 1/18
Number of Bedrooms: 0br
Neighborhood: richmond / seacliff
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-th-beautiful-studio-gift/7579741886.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'attached garage', 'rent period: monthly']

Listing Number: 21
Square Footage: not available
Price: $2,895
Title: Laguna@Waller//Remodeled 1BR
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: hayes valley
Number of Bathrooms: 1
Link: https://sfbay.craigsl


Listing Number: 38
Square Footage: not available
Price: $2,395
Title: renovated 1 bedroom, Gated, Bustops
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: mission district
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-renovated-bedroom-gated/7573956616.html
Amenities: ['apartment', 'no laundry on site', 'street parking', 'rent period: monthly']

Listing Number: 39
Square Footage: not available
Price: $4,995
Title: 5 bedrooms, renovated, parking, Nice , Near hospital
Date: 1/18
Number of Bedrooms: 5br
Neighborhood: potrero hill
Number of Bathrooms: 3
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-bedrooms-renovated/7573472917.html
Amenities: ['apartment', 'laundry in bldg', 'off-street parking', 'rent period: monthly']

Listing Number: 40
Square Footage: not available
Price: $2,850
Title: SHOW SAT 1/21@1PM 1BR/1BA Top Floor Inner Sunset w/Parking Included
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: inner sunset / UCSF
Number of Bathr


Listing Number: 57
Square Footage: not available
Price: $2,595
Title: Remodeled Marina 1BR+Den! $0 Deposit - 1 Month Free; Unit W/D!
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: marina / cow hollow
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-remodeled-marina-1brden/7579734017.html
Amenities: ['application fee details: $23 per applicant', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly']

Listing Number: 58
Square Footage: not available
Price: $3,895
Title: $0 Deposit - Russian Hill Remodeled 2BR/2BA! *1 Month Free* Unit W/D!
Date: 1/18
Number of Bedrooms: 2br
Neighborhood: russian hill
Number of Bathrooms: 2
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-deposit-russian-hill/7579733885.html
Amenities: ['application fee details: $23 per applicant', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent per


Listing Number: 76
Square Footage: 1200
Price: $5,295
Title: Remodeled Cole Valley 3BR/2BA! *$0 Deposit - 1 Month Free* Unit W/D!
Date: 1/18
Number of Bedrooms: 3br
Neighborhood: cole valley / ashbury hts
Number of Bathrooms: 2
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-remodeled-cole-valley-3br/7579725690.html
Amenities: ['application fee details: $23 per applicant', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly']

Listing Number: 77
Square Footage: 1400
Price: $4,995
Title: 3 Bedroom NEAR DOLORES PARK, 1ST MONTH FREE! LIGHT, LAUNDRY, CLOSETS
Date: 1/18
Number of Bedrooms: 3br
Neighborhood: mission district
Number of Bathrooms: split
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-bedroom-near-dolores-park/7577916661.html
Amenities: ['cats are OK - purrr', 'apartment', 'w/d in unit', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 78
Square Footage: not av

...


Listing Number: 32
Square Footage: not available
Price: $4,595
Title: Newly Renovated 3BR w/ In-Unit W/D & Parking | Near GG Park!
Date: 1/18
Number of Bedrooms: 3br
Neighborhood: inner sunset / UCSF
Number of Bathrooms: 2
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-newly-renovated-3br-in/7575255319.html
Amenities: ['apartment', 'w/d in unit', 'no smoking', 'attached garage', 'rent period: monthly']

Listing Number: 33
Square Footage: not available
Price: $4,495
Title: Bright & Spacious Pac Heights 2BR + Formal Dining Room!
Date: 1/18
Number of Bedrooms: 2br
Neighborhood: pacific heights
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-bright-spacious-pac/7576951253.html
Amenities: ['apartment', 'laundry on site', 'no smoking', 'no parking', 'rent period: monthly']

Listing Number: 34
Square Footage: not available
Price: $1,695
Title: Spacious Studio w/ Walk-In-Closet & Eat-In-Kitchen!
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: ten


Listing Number: 51
Square Footage: 800
Price: $3,596
Title: Top Floor (City Views) 800 sqft 1BD/1BA in Luxury South Beach Building
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: SOMA / south beach
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-top-floor-city-views-800/7579508377.html
Amenities: ['application fee details: $50 per occupant', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 52
Square Footage: not available
Price: $3,995
Title: Top Floor! Remodeled 2 Bed with W/D in Unit. 520 Grant #5
Date: 1/18
Number of Bedrooms: 2br
Neighborhood: financial district
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-top-floor-remodeled-bed/7576083034.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 53
Square Footage: not av


Listing Number: 70
Square Footage: not available
Price: $2,395
Title: Newly Updated Studio Unit, W/D in Unit | 1163 Pine #6
Date: 1/18
Number of Bedrooms: 0br
Neighborhood: lower nob hill
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-newly-updated-studio-unit/7576093658.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'no parking', 'rent period: monthly']

Listing Number: 71
Square Footage: not available
Price: $2,695
Title: 1BR Unit in the Heart of SOMA! 906 Howard #301
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: city of san francisco
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-1br-unit-in-the-heart-of/7576091311.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no parking', 'rent period: monthly']

Listing Number: 72
Square Footage: not available
Price: $3,895
Title: Furnished and Updated 1 Bedroom | 720 


Listing Number: 89
Square Footage: 1000
Price: $5,000
Title: Beautiful and Spacious Furnished Four Bedroom Apartment
Date: 1/18
Number of Bedrooms: 5br
Neighborhood: mission district
Number of Bathrooms: 1.5
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-beautiful-and-spacious/7579511472.html
Amenities: ['furnished', 'apartment', 'no laundry on site', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 90
Square Footage: 472
Price: $2,490
Title: Conference rooms, Oversize walk-in closets (select floorplans only)
Date: 1/18
Number of Bedrooms: 0br
Neighborhood: SOMA / south beach
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-conference-rooms-oversize/7579524711.html
Amenities: ['apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 91
Square Footage: not available
Price: $1,995
Title: OPEN TUES 6-6:15PM Studio Sh Laundry(650 Diamond #2)
Date: 1/18
Number of Bedrooms: 0br
Neighborhood: 


Listing Number: 108
Square Footage: not available
Price: $1,950
Title: Upgraded Studio Unit Near Delores Park
Date: 1/18
Number of Bedrooms: 0br
Neighborhood: mission district
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-upgraded-studio-unit-near/7575728815.html
Amenities: ['application fee details: $40 per applicant', 'apartment', 'laundry in bldg', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 109
Square Footage: not available
Price: $2,995
Title: Laurel Heights/99Lupine&Masonic/Great Views/Available now!
Date: 1/18
Number of Bedrooms: 1br
Neighborhood: laurel hts / presidio
Number of Bathrooms: 1
Link: https://sfbay.craigslist.org/sfc/apa/d/san-francisco-laurel-heights/7579518795.html
Amenities: ['application fee details: $35', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'street parking', 'rent period: monthly']

Listing Number: 110
Square Footage: 704
Price: $3,610
Title: Controlled A

In [4]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/18,EPIC REA -840 Sq ft Fresh Paint/New Flooring-2...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$3,300",2br,NaN,potrero hill,1,[application fee details: $45 - tenant screeni...
1,1/18,Golden Gate Views,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$3,500",2br,NaN,russian hill,2,"[apartment, laundry in bldg, no smoking, carpo..."
2,1/18,House-like condo with view and south-facing ya...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$16,500",3br,NaN,laurel hts / presidio,3.5,"[cats are OK - purrr, dogs are OK - wooof, con..."
3,1/18,SPACIOUS MURAL APARTMENT IN HOT MISSION,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$4,350",4br,1650,Inner Mission,1.5,"[apartment, w/d in unit, street parking, rent ..."
4,1/18,EPIC REA - Bright/Beautiful 1 BD/1 BA with Par...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$2,295",1br,442,visitacion valley,1,[application fee details: $45 Tenant Screening...
...,...,...,...,...,...,...,...,...,...
955,1/18,Fully Renovated 2BR 2BA AVAILABLE NOW $3995,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$3,995",2br,NaN,nob hill,2,"[apartment, w/d in unit, no smoking, street pa..."
956,1/18,Fully Renovated 2BR 2BA AVAILABLE NOW $3995,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$3,995",2br,NaN,nob hill,2,"[apartment, w/d in unit, no smoking, street pa..."
957,1/18,Bright top-floor rent-controlled 1-bed 1-bath ...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$2,795",1br,NaN,richmond / seacliff,1,"[apartment, laundry in bldg, attached garage, ..."
958,1/18,4br/2ba @ NOPA! ►REMODELED Victorian Flat!,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,"$5,499",4br,NaN,USF / panhandle,2,"[cats are OK - purrr, dogs are OK - wooof, fla..."


In [5]:
# Exporting data to csv
df.to_csv(f"{city.replace(' ', '_')}_data_raw.csv", index=False)